In [12]:
!pip install plotly
import requests
import pandas as pd

URL = 'https://pokeapi.co/api/v2/pokemon/'

In [2]:
def fetch_pokemon(pokemon_id):
    url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_id}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao acessar {url}: {response.status_code}")
        return None

# Vamos coletar os dados de um conjunto de Pokémon para demonstração
pokemon_data = []
for i in range(1, 200):  # Coletar dados de 20 Pokémon para evitar sobrecarga
    data = fetch_pokemon(i)
    if data:
        pokemon_data.append(data)

In [3]:
def process_data(raw_data):
    processed_data = {
        'id': raw_data['id'],
        'name': raw_data['name'],
        'type': [type['type']['name'] for type in raw_data['types']],
        'hp': raw_data['stats'][0]['base_stat'],
        'attack': raw_data['stats'][1]['base_stat'],
        'defense': raw_data['stats'][2]['base_stat']
    }
    return processed_data

# Criar uma lista de dicionários com dados processados
pokemon_list = [process_data(pokemon) for pokemon in pokemon_data]

# Converter a lista de dicionários em um DataFrame
df_pokemon = pd.DataFrame(pokemon_list)
df_pokemon

,id,name,type,hp,attack,defense
0,1,bulbasaur,"[grass, poison]",45,49,49
1,2,ivysaur,"[grass, poison]",60,62,63
2,3,venusaur,"[grass, poison]",80,82,83
3,4,charmander,[fire],39,52,43
4,5,charmeleon,[fire],58,64,58
...,...,...,...,...,...,...
194,195,quagsire,"[water, ground]",95,85,85
195,196,espeon,[psychic],65,65,60
196,197,umbreon,[dark],95,65,110
197,198,murkrow,"[dark, flying]",60,85,42


In [4]:
df_pokemon_exploded = df_pokemon.explode('type')
print(df_pokemon_exploded.head())

   id       name    type  hp  attack  defense
0   1  bulbasaur   grass  45      49       49
0   1  bulbasaur  poison  45      49       49
1   2    ivysaur   grass  60      62       63
1   2    ivysaur  poison  60      62       63
2   3   venusaur   grass  80      82       83


In [8]:
# Agrupar por tipo e encontrar o Pokémon com o maior HP e Defesa em cada tipo
strongest_hp_per_type = df_pokemon_exploded.loc[df_pokemon_exploded.groupby('type')['hp'].idxmax()]
strongest_defense_per_type = df_pokemon_exploded.loc[df_pokemon_exploded.groupby('type')['defense'].idxmax()]

# Selecione globalmente os top 10 de HP e Defesa
top_10_hp = strongest_hp_per_type.nlargest(10, 'hp')
top_10_defense = strongest_defense_per_type.nlargest(10, 'defense')

print(top_10_hp[['name', 'type', 'hp']])
print(top_10_defense[['name', 'type', 'defense']])

           name      type   hp
112     chansey    normal  250
39   wigglytuff    normal  140
39   wigglytuff     fairy  140
130      lapras     water  130
130      lapras       ice  130
130      lapras     water  130
130      lapras       ice  130
170     lanturn     water  125
170     lanturn  electric  125
149      mewtwo   psychic  106
         name    type  defense
90   cloyster   water      180
90   cloyster     ice      180
90   cloyster   water      180
90   cloyster     ice      180
94       onix    rock      160
94       onix  ground      160
94       onix    rock      160
94       onix  ground      160
109   weezing  poison      120
113   tangela   grass      115


In [17]:
import plotly.graph_objects as go

# Gráfico para os 10 Pokémon mais fortes em HP
fig_hp = go.Figure()
fig_hp.add_trace(go.Bar(
    x=top_10_hp['name'],
    y=top_10_hp['hp'],
    marker_color='green',
    text=top_10_hp['type'],  # Mostrar o tipo no gráfico
    textposition='auto'
))

fig_hp.update_layout(
    title='Top 10 Pokémon Mais Fortes por HP',
    xaxis_title="Nome do Pokémon",
    yaxis_title="HP",
    template='plotly_white',
    xaxis={'categoryorder':'total descending'}
)

# Gráfico para os 10 Pokémon mais fortes em Defesa
fig_defense = go.Figure()
fig_defense.add_trace(go.Bar(
    x=top_10_defense['name'],
    y=top_10_defense['defense'],
    marker_color='blue',
    text=top_10_defense['type'],  # Mostrar o tipo no gráfico
    textposition='auto'
))

fig_defense.update_layout(
    title='Top 10 Pokémon Mais Fortes por Defesa',
    xaxis_title="Nome do Pokémon",
    yaxis_title="Defesa",
    template='plotly_white',
    xaxis={'categoryorder':'total descending'}
)

# Exibir os gráficos
fig_hp.show()
fig_defense.show()

In [18]:
# Encontrar o Pokémon com o maior ataque em cada tipo
strongest_attack_per_type = df_pokemon_exploded.loc[df_pokemon_exploded.groupby('type')['attack'].idxmax()]

# Se você deseja ver apenas os top 10 em geral depois de encontrar o mais forte de cada tipo:
top_10_attack = strongest_attack_per_type.nlargest(10, 'attack')

In [19]:
import plotly.graph_objects as go

# Gráfico para os 10 Pokémon mais fortes em Ataque
fig_attack = go.Figure()
fig_attack.add_trace(go.Bar(
    x=top_10_attack['name'],
    y=top_10_attack['attack'],
    marker_color='red',  # Cor vermelha para destacar o ataque
    text=top_10_attack['type'],  # Mostrar o tipo de cada Pokémon
    textposition='auto'
))

fig_attack.update_layout(
    title='Top 10 Pokémon Mais Fortes por Ataque',
    xaxis_title="Nome do Pokémon",
    yaxis_title="Ataque",
    template='plotly_white',
    xaxis={'categoryorder':'total descending'}
)

# Exibir o gráfico
fig_attack.show()

In [21]:
# Salvar o DataFrame de HP como CSV
strongest_hp_per_type.to_csv('strongest_hp_per_type.csv', index=False)

# Salvar o DataFrame de Defesa como CSV
strongest_defense_per_type.to_csv('strongest_defense_per_type.csv', index=False)

# Salvar o DataFrame de Ataque como CSV (já mencionado anteriormente)
strongest_attack_per_type.to_csv('strongest_attack_per_type.csv', index=False)